# Portfolio Optimization 

In [1]:
import requests
import time
import numpy as np
import pandas as pd

# Gather Data from Yahoo for SPLG and GLD
    - Add other symbols as a list

In [58]:
import yfinance as yf

def get_stock_data(symbols=[], start_year=2000, get_benchmark=True):
    if get_benchmark:
      benchmark = ['SPLG', 'GLD', 'SH'] 
      # add benchmark to the list of symbols
      symbols = benchmark + symbols
    # get adj close for the list of tickers
    today = pd.Timestamp.today().strftime('%Y-%m-%d')
    df = yf.download(symbols, start=f'{start_year}-01-01' , end=today)['Adj Close']
    return df

In [3]:
from pypfopt import risk_models, expected_returns, get_latest_prices
from pypfopt.discrete_allocation import DiscreteAllocation
from pypfopt.efficient_frontier import EfficientFrontier

def optimize_portfolio(df, return_model='mean_historical', risk_model=0, optimization_type='max_sharpe', target_risk=None, target_return=None, print_weights_dataframe=False, span=252):
    '''
    Optimizes a portfolio for the maximum Sharpe ratio or a target risk/return, using specified methods for estimating returns and risk.

    Parameters:
    df (DataFrame): DataFrame containing historical asset prices or returns.
    optimization_type (str): Type of optimization ('max_sharpe', 'min_volatility', 'target_risk', 'target_return').
    target_risk (float): Target risk level (required if optimization_type is 'target_risk').
    target_return (float): Target return level (required if optimization_type is 'target_return').
    risk_model (int): Index of the risk model to be used from the risk_models_list.
    return_model (str): Method for estimating expected returns ('mean_historical', 'ema_historical', 'capm').
    print_weights_dataframe (bool): If True, prints the dataframe of portfolio weights.
    span (int): The time span in days to be considered for calculating EMA Historical Returns.
                The default value is 252, typically representing the number of trading days in a year.
                This parameter is only used when 'ema_historical' is selected as the return_model.

    Most Common Combinations of Return Estimation Methods and Risk Models:
    1. Mean Historical Returns and Sample Covariance:
       - Mean Historical Returns: Calculates the average return of each asset over the historical period.
       - Sample Covariance: Estimates the covariance matrix based on historical returns of the assets.
       - Common Use: Ideal for simplicity and traditional mean-variance optimization.
       - Parameters: return_model='mean_historical', risk_model=0 (for sample_cov)

    2. Exponential Moving Average (EMA) Historical Returns and Exponential Covariance:
       - EMA Historical Returns: Gives more weight to recent returns, assuming they are more indicative of future performance.
       - Exponential Covariance: Similar to EMA returns, it weighs recent price movements more heavily in estimating the covariance matrix.
       - Common Use: Suitable for dynamic markets or when recent trends are considered more relevant.
       - Parameters: return_model='ema_historical', risk_model=1 (for exp_cov)

    3. CAPM Returns and Ledoit-Wolf Shrinkage:
       - CAPM Returns: Uses the CAPM model to estimate expected returns based on market expectations and systematic risk.
       - Ledoit-Wolf Shrinkage: Improves covariance estimation by reducing extreme values towards the average.
       - Common Use: Balances theoretical rigor with practical adjustments for estimation error.
       - Parameters: return_model='capm', risk_model=3 (for ledoit_wolf)

    Returns:
    dict: Cleaned portfolio weights after optimization.
    '''

    # Define return estimation methods
    return_models = {
        'mean_historical': expected_returns.mean_historical_return,
        'ema_historical': lambda x: expected_returns.ema_historical_return(x, span=span),
        'capm': expected_returns.capm_return,
    }

    # Validate and apply the chosen return estimation method
    if return_model in return_models:
        mu = return_models[return_model](df)
    else:
        raise ValueError(f"Invalid return model specified: {return_model}. Choose from {list(return_models.keys())}")

    # Define risk models
    risk_models_list = [
        ('sample_cov', risk_models.sample_cov),
        ('exp_cov', lambda x: risk_models.exp_cov(x, span=span)),
        ('semicovariance', lambda x: risk_models.risk_matrix(x, method='semicovariance', benchmark=0, frequency=252)),
        ('ledoit_wolf', lambda x: risk_models.risk_matrix(x, method='ledoit_wolf')),
        ('ledoit_wolf_constant_variance', lambda x: risk_models.risk_matrix(x, method='ledoit_wolf_constant_variance')),
        ('ledoit_wolf_single_factor', lambda x: risk_models.risk_matrix(x, method='ledoit_wolf_single_factor')),
        ('ledoit_wolf_constant_correlation', lambda x: risk_models.risk_matrix(x, method='ledoit_wolf_constant_correlation')),
    ]

    try:
        name, model = risk_models_list[risk_model]
        Sigma = model(df)

        ef = EfficientFrontier(mu, Sigma)

        if optimization_type == 'max_sharpe':
            raw_weights = ef.max_sharpe()
        elif optimization_type == 'min_volatility':
            raw_weights = ef.min_volatility()
        elif optimization_type == 'target_risk':
            if target_risk is not None:
                raw_weights = ef.efficient_risk(target_risk)
            else:
                raise ValueError("Target risk not specified for 'target_risk' optimization.")
        elif optimization_type == 'target_return':
            if target_return is not None:
                raw_weights = ef.efficient_return(target_return)
            else:
                raise ValueError("Target return not specified for 'target_return' optimization.")
        else:
            raise ValueError("Invalid optimization type specified.")


        cleaned_weights = ef.clean_weights()
        # get rid of any weights that are 0 or negative
        cleaned_weights = {k: v for k, v in cleaned_weights.items() if v > 0}
        if print_weights_dataframe:
          print('[CLEANED WEIGHTS]')
          cleaned_weights_df = pd.DataFrame(list(cleaned_weights.items()), columns=['Ticker', 'Value'])
          display(cleaned_weights_df)

        performance = ef.portfolio_performance(verbose=True)
        print(f'Performance using {name}:', performance, end='\n\n')
        return cleaned_weights
    except Exception as e:
        print(f"An error occurred with the {name} model: {e}")


In [18]:
import plotly.graph_objects as go
import matplotlib.pyplot as plt
plt.style.use('bmh') # other styles are 'ggplot', 'fivethirtyeight', 'dark_background', 'seaborn', 'seaborn-whitegrid'

def explain_correlation():
  print(
    'Correlation Explained:\n'
    '\t- Value Range: Correlation values range from -1 to +1.\n'
    '\t- Positive Value (0 to +1): Indicates a positive relationship between the returns of two stocks.\n\t  As one stock\'s return increases, the other\'s tends to increase as well.\n\t  A value closer to +1 signifies a stronger positive correlation.\n'
    '\t- Negative Value (-1 to 0): Suggests a negative relationship.\n\t  As one stock\'s return increases, the other\'s tends to decrease.\n\t  A value closer to -1 indicates a stronger negative correlation.\n'
    '\t- Value Near Zero: Implies little to no linear relationship between the returns of the two stocks.\n'
    '\t- Value of +1 or -1: Indicates a perfect positive or negative linear relationship, respectively.\n'
    '\t- Interpretation: High positive correlation might suggest similar factors affect the stocks similarly,\n\t  while high negative correlation could indicate opposite reactions to the same factors.\n'
    '\t- Importance: Understanding correlation helps in diversifying a portfolio.\n\t  Stocks with low or negative correlation can reduce overall portfolio risk.\n'
    '\t- Caution: Correlation does not imply causation.\n\t  Two stocks may move together or inversely without one directly causing the other\'s movement.'
    )

def explain_covariance():
  print(
    'Covariance Explained:\n'
    '\t- Positive Value: Indicates that the returns of two stocks tend to move in the same direction.\n'
    '\t- Negative Value: Suggests that the returns of the two stocks tend to move in opposite directions.\n'
    '\t- Value Near Zero: Implies a very weak or no linear relationship between the returns of the two stocks.\n'
    '\t- Higher Absolute Value: Signifies a stronger relationship between the stock movements.\n\t  The higher the number (positive or negative), the stronger the relationship.\n'
    '\t- Interpretation: Positive covariance can suggest that stocks could be used together for a growth-focused strategy,\n\t  while negative covariance could be used for hedging.\n'
    '\t- Limitation: Covariance alone does not indicate the magnitude of movements;\n\t  it should be considered alongside other metrics like correlation coefficients and volatility.'
  )

def compute_daily_returns(stock_portfolio, times_by_100= True):
    '''Calculation
    df[t] / df[t-1] - 1
    where t is the current time(today) and t-1 is the previous time(yesterday
    for simplicity we will use pct_change() method
    '''
    # calculate daily returns
    if times_by_100:
        daily_ret = stock_portfolio.pct_change(1).mul(100)
    else:
        daily_ret = stock_portfolio.pct_change(1)
    return daily_ret

def cumulative_returns(stock_portfolio):
    # show cumulative returns
    cum_returns = ((stock_portfolio / stock_portfolio.iloc[0])-1) * 100
    return cum_returns

def show_portfolio(stock_portfolio, col='Adj Close', normalize=False, fig_width=1200, fig_height=600):
    if normalize:
        # normalize the data
        stock_portfolio = (stock_portfolio / stock_portfolio.iloc[0]) - 1
    # Create a Plotly graph
    fig = go.Figure()
    for stock in stock_portfolio.columns:
        fig.add_trace(go.Scatter(x=stock_portfolio.index, y=stock_portfolio[stock], mode='lines', name=stock))
    # Update layout
    fig.update_layout(
        title=f'Portfolio {col}',
        xaxis_title='',
        yaxis_title=col,
        legend_title='Stocks',
        width=fig_width,
        height=fig_height
    )
    # Show plot
    fig.show()

def visualize_data(visualize_data):
    show_portfolio(visualize_data, col='Adj Close Normalized', normalize=True)
    cum_returns = cumulative_returns(visualize_data)
    show_portfolio(cum_returns, 'Cumulative Returns')
    daily_returns = compute_daily_returns(visualize_data)
    print('\n[Daily Returns Correlation]')
    display(daily_returns.corr(method='pearson').style.background_gradient(cmap='coolwarm').format(precision=2))
    explain_correlation()
    print('\n[Daily Returns Covariance]')
    display(daily_returns.cov().style.background_gradient(cmap='coolwarm').format(precision=2))
    explain_covariance()
    print("\n[The Portfolio's Volitility]")
    display(daily_returns.std().sort_values(ascending=False).to_frame('Volatility(STD)').style.background_gradient(cmap='coolwarm').format(precision=2))
    print('The higher the volatility, the riskier the security.\nThe lower the volatility, the safer the security')

# Running Portfolio Analysis for:
    - SPLG and GLD

    - GOLD ETFs
            SSgA SPDR Gold Shares (GLD) YIELD = 0    
            Sprott Gold Miners ETF (SGDM) YIELD = 1.45%  
            VanEck Vectors Gold Miners ETF (GDX) YIELD = 1.68%   
            iShares MSCI Global Gold Miners ETF (RING) YIELD = 2.10%   
            VanEck Vectors Junior Gold Miners ETF (GDXJ) YIELD = 0.76%   

    - S&P ETFs:
            SPDR Portfolio S&P 500 ETF (SPLG) YIELD = 1.46%   
            SPDR S&P Dividend ETF (SDY) YIELD = 2.66%   
            iShares Select Dividend ETF (DVY) YIELD = 3.79%   
            Vanguard Dividend Appreciation ETF (VIG) YIELD = 1.90%
            Vanguard Value ETF (VTV) YIELD = 2.45%
            CSIM Schwab U.S. Large-Cap Value ETF (SCHV) YIELD = 2.43%
            SPDR Portfolio S&P 500 High Dividend ETF (SPYD) YIELD = 4.63%
            CSIM Schwab US Dividend Equity ETF (SCHD) YIELD = 3.48%
            Vanguard High Dividend Yield Indx ETF (VYM) YIELD = 3.11%
            BTC iShares Core Dividend Growth ETF (DGRO) YIELD = 2.45%
            Vanguard International High Dividend Yield ETF (VYMI) YIELD = 4.60%
            Vanguard Russell 1000 Value Index ETF (VONV) YIELD = 2.10%
            Vanguard Short-Term Corporate Bond ETF (VCSH) YIELD = 3.11%
            Vanguard Short-Term Treasury ETF (VGSH) YIELD = 3.32%
            Vanguard Short-Term Inflation-Protected Securities Index Fund (VTIP) YIELD = 2.86%
            Vanguard Whitehall - Emerging Market Bond ETF (VWOB) YIELD = 5.63%
            Vanguard Total Corporate Bond ETF (VTC) YIELD = 3.86%
            Vanguard Long-Term Treasury ETF (VGLT) YIELD = 3.41%
            ProShares Short S&P500 (SH) YIELD = 5.28%
            

In [61]:
#  SPLG and GLD are the benchmark ETFs for the S&P 500 and Gold and are already included in the list of symbols in the get_stock_data() function
GOLD_ETFs = ['SGDM', 'GDX', 'RING', 'GDXJ']
S_P500_ETFs = ['SH', 'SDY', 'DVY', 'VIG', 'VTV', 'SCHV', 'SPYD', 'SCHD', 'VYM', 'DGRO', 'VYMI', 'VONV', 'VCSH', 'VGSH', 'VTIP', 'VWOB', 'VTC', 'VGLT']

COMBINED = GOLD_ETFs + S_P500_ETFs

splg_gld_data = get_stock_data(symbols=[], get_benchmark=True)
# drop na values
splg_gld_data.dropna(inplace=True, axis=0) # axis 1 is for dropping columns, axis 0 is  for dropping rows
splg_gld_data

[*********************100%***********************]  3 of 3 completed


,GLD,SH,SPLG
Date,,,
2006-06-21,58.330002,102.860237,10.191137
2006-06-22,57.730000,103.677910,10.252337
2006-06-23,57.990002,103.707634,10.261333
2006-06-26,58.279999,103.380569,10.286532
2006-06-27,57.680000,104.272583,10.205538
...,...,...,...
2023-12-29,191.169998,12.990000,55.900002
2024-01-02,190.720001,13.070000,55.599998
2024-01-03,189.130005,13.170000,55.150002


In [8]:
# show the Help for the optimize_portfolio() function
optimize_portfolio?

Signature:
optimize_portfolio(
    df,
    return_model='mean_historical',
    risk_model=0,
    optimization_type='max_sharpe',
    target_risk=None,
    target_return=None,
    print_weights_dataframe=False,
    span=252,
)
Docstring:
Optimizes a portfolio for the maximum Sharpe ratio or a target risk/return, using specified methods for estimating returns and risk.

Parameters:
df (DataFrame): DataFrame containing historical asset prices or returns.
optimization_type (str): Type of optimization ('max_sharpe', 'min_volatility', 'target_risk', 'target_return').
target_risk (float): Target risk level (required if optimization_type is 'target_risk').
target_return (float): Target return level (required if optimization_type is 'target_return').
risk_model (int): Index of the risk model to be used from the risk_models_list.
return_model (str): Method for estimating expected returns ('mean_historical', 'ema_historical', 'capm').
print_weights_dataframe (bool): If True, prints the dataframe 

In [63]:
# 3 most common combinations of return estimation methods and risk models
# optimize_portfolio(splg_gld_data, return_model='mean_historical', risk_model=0)
ema_weights = optimize_portfolio(splg_gld_data, return_model='ema_historical', risk_model=1, span=252, print_weights_dataframe=True)
# optimize_portfolio(splg_gld_data, return_model='capm', risk_model=3)
print(ema_weights, end='\n\n')
# semi cov optimization
print('[Semi Cov Optimization]')
semi_cov_weights = optimize_portfolio(splg_gld_data, return_model='mean_historical', risk_model=2)
# optimization along with min_vol
print('[Optimization with min_volatility]')
min_volatility_weights = optimize_portfolio(splg_gld_data, return_model='mean_historical', risk_model=0, optimization_type='min_volatility')
# optimize_portfolio(splg_gld_data, return_model='ema_historical', risk_model=1, optimization_type='min_volatility')

[CLEANED WEIGHTS]


,Ticker,Value
0,GLD,0.00380
1,SH,0.49623
2,SPLG,0.49997


Expected annual return: 5.6%
Annual volatility: 0.5%
Sharpe Ratio: 6.76
Performance using exp_cov: (0.055888199740471, 0.005312704636421367, 6.755165625892062)

{'GLD': 0.0038, 'SH': 0.49623, 'SPLG': 0.49997}

[Semi Cov Optimization]
Expected annual return: 9.0%
Annual volatility: 11.1%
Sharpe Ratio: 0.64
Performance using semicovariance: (0.09044283838415558, 0.11069424212105161, 0.6363731033735394)

[Optimization with min_volatility]
Expected annual return: 0.2%
Annual volatility: 4.4%
Sharpe Ratio: -0.41
Performance using sample_cov: (0.0020240011886984363, 0.04357966002227481, -0.4124859808936902)



In [64]:
# Filter the DataFrame with only the symbols that have assigned weights in the portfolio
symbols_list = list(ema_weights.keys())

# Set the total amount to be invested in the portfolio
portfolio_investment = 50000

# Get the latest prices of the symbols from the filtered DataFrame
latest_prices = get_latest_prices(splg_gld_data[symbols_list])

# Create a DiscreteAllocation object with the weights, latest prices, and total investment value
# DiscreteAllocation helps in determining the number of each stock to buy given the weights and available capital
da = DiscreteAllocation(ema_weights, latest_prices, total_portfolio_value=portfolio_investment)

# Perform the portfolio optimization using a linear programming approach
# Returns the optimal allocation and the leftover funds after purchasing
allocation, leftover = da.lp_portfolio()

# Print out the total portfolio investment and the leftover funds
print(f"Discrete allocation performed with ${portfolio_investment:.2f}")
print(f"Leftover: ${leftover:.2f}")

# Convert the allocation dictionary to a DataFrame for better visualization
# The DataFrame shows the number of shares for each symbol in the portfolio
allocation_df = pd.DataFrame.from_dict(allocation, orient='index', columns=['Shares'])

# Display the allocation DataFrame
display(allocation_df)


Discrete allocation performed with $50000.00
Leftover: $5.03


,Shares
GLD,1
SH,1878
SPLG,454


In [65]:
# adjust the shares for fractional buying based on decreasing the investment amount
division_factor = 75
lower_investment =  portfolio_investment / division_factor
print("The new fractional shares are calculated based on a",  f"reduced portfolio investment value of ${round(lower_investment, 2)}.", sep='\n', end='\n\n')
allocation_df['fract_shares'] = allocation_df.Shares.div(division_factor)

display(allocation_df)

The new fractional shares are calculated based on a
reduced portfolio investment value of $666.67.



,Shares,fract_shares
GLD,1,0.013333
SH,1878,25.040000
SPLG,454,6.053333


In [66]:

# show actual prices
show_portfolio(splg_gld_data[symbols_list], col='Ajusted Close Prices', normalize=False)
# visualize the data
visualize_data(splg_gld_data[symbols_list])



[Daily Returns Correlation]


,GLD,SH,SPLG
GLD,1.00,-0.05,0.05
SH,-0.05,1.00,-0.89
SPLG,0.05,-0.89,1.00


Correlation Explained:
	- Value Range: Correlation values range from -1 to +1.
	- Positive Value (0 to +1): Indicates a positive relationship between the returns of two stocks.
	  As one stock's return increases, the other's tends to increase as well.
	  A value closer to +1 signifies a stronger positive correlation.
	- Negative Value (-1 to 0): Suggests a negative relationship.
	  As one stock's return increases, the other's tends to decrease.
	  A value closer to -1 indicates a stronger negative correlation.
	- Value Near Zero: Implies little to no linear relationship between the returns of the two stocks.
	- Value of +1 or -1: Indicates a perfect positive or negative linear relationship, respectively.
	- Interpretation: High positive correlation might suggest similar factors affect the stocks similarly,
	  while high negative correlation could indicate opposite reactions to the same factors.
	- Importance: Understanding correlation helps in diversifying a portfolio.
	  Stocks with l

,GLD,SH,SPLG
GLD,1.22,-0.07,0.07
SH,-0.07,1.57,-1.33
SPLG,0.07,-1.33,1.42


Covariance Explained:
	- Positive Value: Indicates that the returns of two stocks tend to move in the same direction.
	- Negative Value: Suggests that the returns of the two stocks tend to move in opposite directions.
	- Value Near Zero: Implies a very weak or no linear relationship between the returns of the two stocks.
	- Higher Absolute Value: Signifies a stronger relationship between the stock movements.
	  The higher the number (positive or negative), the stronger the relationship.
	- Interpretation: Positive covariance can suggest that stocks could be used together for a growth-focused strategy,
	  while negative covariance could be used for hedging.
	- Limitation: Covariance alone does not indicate the magnitude of movements;
	  it should be considered alongside other metrics like correlation coefficients and volatility.

[The Portfolio's Volitility]


,Volatility(STD)
SH,1.25
SPLG,1.19
GLD,1.11


The higher the volatility, the riskier the security.
The lower the volatility, the safer the security


In [ ]:
from pypfopt import risk_models, expected_returns, get_latest_prices
from pypfopt.discrete_allocation import DiscreteAllocation
from pypfopt.efficient_frontier import EfficientFrontier
import requests
import time
import numpy as np
import yfinance as yf
import pandas as pd
import plotly.graph_objects as go 
import matplotlib.pyplot as plt
plt.style.use('bmh') # other styles are 'ggplot', 'fivethirtyeight', 'dark_background', 'seaborn', 'seaborn-whitegrid'

def find_key_by_value(rm, value_to_find):
    for key, value in rm.items():
        if value == value_to_find:
            return key, value
    return None, None  # Return None if no match is found

def visualize_data(visualize_data):
    show_portfolio(visualize_data, normalize=True)
    cum_returns = cumulative_returns(visualize_data)
    show_portfolio(cum_returns, 'Cumulative Returns')
    daily_returns = compute_daily_returns(visualize_data)
    print('[Daily Returns Correlation]')
    display(daily_returns.corr(method='pearson').style.background_gradient(cmap='coolwarm').format(precision=2))
    print('\n[Daily Returns Covariance]')
    display(daily_returns.cov().style.background_gradient(cmap='coolwarm').format(precision=2))
    print("\n[The Portfolio's Volitility]")
    display(daily_returns.std().sort_values(ascending=False).to_frame('Volatility(STD)').style.background_gradient(cmap='coolwarm').format(precision=2))
    print('The higher the volatility, the riskier the security.\nThe lower the volatility, the safer the security')

def compute_daily_returns(stock_portfolio, times_by_100= True):
    '''Calculation
    df[t] / df[t-1] - 1
    where t is the current time(today) and t-1 is the previous time(yesterday
    for simplicity we will use pct_change() method
    '''
    # calculate daily returns
    if times_by_100:
        daily_ret = stock_portfolio.pct_change(1).mul(100)
    else:
        daily_ret = stock_portfolio.pct_change(1)
    return daily_ret

def cumulative_returns(stock_portfolio):
    # show cumulative returns
    cum_returns = ((stock_portfolio / stock_portfolio.iloc[0])-1) * 100
    return cum_returns

def show_portfolio(stock_portfolio, col='Adj Close', normalize=False):
    if normalize:
        # normalize the data
        stock_portfolio = (stock_portfolio / stock_portfolio.iloc[0]) - 1
    # Create a Plotly graph
    fig = go.Figure()
    for stock in stock_portfolio.columns:
        fig.add_trace(go.Scatter(x=stock_portfolio.index, y=stock_portfolio[stock], mode='lines', name=stock))
    # Update layout
    fig.update_layout(
        title=f'Portfolio {col}',
        xaxis_title='Date',
        yaxis_title=col,
        legend_title='Stocks',
        margin=dict(l=0, r=0, t=30, b=0)
    )
    # Show plot
    fig.show()

def remove_columns_with_few_data_points(df, min_data_points=150):
    """
    Remove columns from the DataFrame that have fewer than 'min_data_points' non-NaN values.

    Parameters:
    df (pd.DataFrame): The input DataFrame.
    min_data_points (int): The minimum number of non-NaN data points required to keep a column.

    Returns:
    pd.DataFrame: A DataFrame with columns having fewer than 'min_data_points' non-NaN values removed.
    """
    return df.dropna(thresh=min_data_points, axis=1, inplace=True)

def incremental_optimize_portfolio(df, initial_size=10, filename='data\\BEST_PORTFOLIO_STOCKS', risk_model=0):
    total_stocks = df.shape[1]
    best_performance = -float('inf')
    best_portfolio = {}
    selected_symbols = list(df.columns[:initial_size])

    for end in range(initial_size, total_stocks + 1):
        # Optimize using only selected symbols plus one new symbol
        if end < total_stocks:
            symbols_to_optimize = selected_symbols + [df.columns[end]]
        else:
            symbols_to_optimize = selected_symbols

        subset_df = df[symbols_to_optimize]
        print(f"Optimizing for stocks: {symbols_to_optimize}")
        performance, portfolio = optimize_portfolio_max_sharpe(subset_df, risk_model=risk_model)

        if performance > best_performance:
            best_performance = performance
            best_portfolio = portfolio

            # Update selected_symbols with symbols from the best portfolio
            selected_symbols = [symbol for symbol in symbols_to_optimize if portfolio.get(symbol, 0) > 0]

        print('-' * 100)

    # Display the best portfolio
    best_portfolio_df = pd.DataFrame.from_dict(best_portfolio, orient='index', columns=['Weight'])
    print("Best Portfolio:")
    display(best_portfolio_df)
    print(f"Symbols in the best portfolio: {selected_symbols}")
    print(f"Best expected annual return: {best_performance}")
    # save the best portfolio to csv
    best_performance = round(best_performance*100, 2)
    save_to_csv(best_portfolio_df, f'{filename}_{best_performance}%.csv', index=True)


    return selected_symbols, best_performance

def optimize_portfolio_max_sharpe(df, risk_model=0):
    mu = expected_returns.mean_historical_return(df)
    risk_models_list = [
        ('sample_cov', risk_models.sample_cov),
        ('exp_cov', risk_models.exp_cov),
        ('semicovariance', lambda x: risk_models.risk_matrix(x, method='semicovariance')),
        ('ledoit_wolf', lambda x: risk_models.risk_matrix(x, method='ledoit_wolf')),
        ('ledoit_wolf_constant_variance', lambda x: risk_models.risk_matrix(x, method='ledoit_wolf_constant_variance')),
        ('ledoit_wolf_single_factor', lambda x: risk_models.risk_matrix(x, method='ledoit_wolf_single_factor')),
        ('ledoit_wolf_constant_correlation', lambda x: risk_models.risk_matrix(x, method='ledoit_wolf_constant_correlation')),
        ('oracle_approximating', lambda x: risk_models.risk_matrix(x, method='oracle_approximating')),
    ]

    try:
        name, model = risk_models_list[risk_model]
        Sigma = model(df)
        
        ef = EfficientFrontier(mu, Sigma)
        raw_weights = ef.max_sharpe()
        cleaned_weights = ef.clean_weights()
        # get rid of any weights that are 0 or negative
        cleaned_weights = {k: v for k, v in cleaned_weights.items() if v > 0}
        # print('[CLEANED WEIGHTS]')
        # cleaned_weights_df = pd.DataFrame(list(cleaned_weights.items()), columns=['Ticker', 'Value'])
        # display(cleaned_weights_df)

        performance = ef.portfolio_performance(verbose=True)
        # print(f'Performance using {name}:', performance)
        return performance[0], cleaned_weights  # Return expected annual return and portfolio
    except Exception as e:
        print(f"An error occurred with the {name} model: {e}")
        return -float('inf'), None

def convert_to_datetime_index(df):
    # convert date column to datetime
    try:
        df['Date'] = pd.to_datetime(df['Date'], unit='ms').dt.normalize()
    except:    
        df['Date'] = pd.to_datetime(df['Date']).dt.normalize()
    # set date column as index
    df = df.set_index('Date')
    return df

def get_symbols_list(df_column):
    symbols_list = df_column.tolist()
    # remove duplicates
    symbols_list = set(symbols_list)
    return list(symbols_list)

def clean_data(df, ticker):
    # drop all columns exept t and c
    df = df.drop(['v', 'vw', 'o', 'h', 'l', 'n'], axis=1)
    # rename columns
    df = df.rename(columns={'t': 'date', 'c': ticker})
    return df

def get_stock_data(symbols=[], start_year=2000):
    benchmark = ['SPLG', 'GLD']
    # add benchmark to the list of symbols
    symbols = benchmark + symbols
    # get adj close for the list of tickers
    today = pd.Timestamp.today().strftime('%Y-%m-%d')
    df = yf.download(symbols, start=f'{start_year}-01-01' , end=today)['Adj Close']
    return df

def get_all_dividend_stocks(API_KEY):
    '''query all dividend stocks'''
    today = pd.Timestamp.today().strftime('%Y-%m-%d')
    year_ahead = (pd.Timestamp.today() + pd.DateOffset(years=1)).strftime('%Y-%m-%d')
    base_url = f'https://api.polygon.io/v3/reference/dividends?ex_dividend_date.gt={today}&ex_dividend_date.lt={year_ahead}&limit=1000&sort=ex_dividend_date'
    url = f'{base_url}&apiKey={API_KEY}'
    symbols = []

    while True:
        response = requests.get(url)
        # Check for successful response
        if response.status_code != 200:
            print('Bad response from Polygon.io API.')
            print(f'Status Code: {response.status_code}')
            return None

        try:
            data = response.json()
            symbols.extend(data['results'])

            # Check if 'next_url' is present, else break the loop
            if 'next_url' in data and data['next_url']:
                url = data['next_url']
                url = f'{url}&apiKey={API_KEY}'
                print(f'Fetching data from\n{url},\nplease wait...need to wait 15 seconds to avoid rate limit')
                time.sleep(15)
            else:
                break
        except KeyError:
            print("KeyError: 'results' not found in the response.")
            print("Response content:\n", data)  # Log the content of 'data' for debugging
            break  

    # Convert the list of symbols to a DataFrame
    symbols_df = pd.DataFrame(symbols)
    return symbols_df

def get_all_active_stocks(API_KEY, ticker_type, market, exchange):
    '''
    Query all ticker symbols which are supported by Polygon.io
    '''
    base_url = f'https://api.polygon.io/v3/reference/tickers?type={ticker_type}&market={market}&exchange={exchange}&active=true&limit=1000'
    url = f'{base_url}&apiKey={API_KEY}'
    symbols = []
    while True:
        response = requests.get(url)
        # Check for successful response
        if response.status_code != 200:
            print('Bad response from Polygon.io API.')
            print(f'Status Code: {response.status_code}')
            return None
        try:
            data = response.json()
            symbols.extend(data['results'])
            # Check if 'next_url' is present, else break the loop
            if 'next_url' in data and data['next_url']:
                url = data['next_url']
                url = f'{url}&apiKey={API_KEY}'
                print(f'Fetching data from\n{url},\nplease wait...need to wait 15 seconds to avoid rate limit')
                time.sleep(15)
            else:
                break
        except KeyError:
            print("KeyError: 'results' not found in the response.")
            print("Response content:\n", data)  # Log the content of 'data' for debugging
            break    

    # Convert the list of symbols to a DataFrame
    symbols_df = pd.DataFrame(symbols)
    return symbols_df

def get_ticker_types(API_KEY):
    '''
    List all ticker types that Polygon.io has for US stocks
    '''
    url = f'https://api.polygon.io/v3/reference/tickers/types?asset_class=stocks&locale=us&apiKey={API_KEY}'
    response = requests.get(url)
    #  check for successful response
    if response.status_code != 200:
        print('Bad response from Polygon.io API.')
        print(f'Status Code: {response.status_code}')
        return None
    data = response.json()
    # convert to dataframe
    types_df = pd.DataFrame(data['results'])
    return types_df

def load_csv(filename, has_date_column=False):
    df = pd.read_csv(filename)
    if has_date_column:
        df = convert_to_datetime_index(df)
    return df

def save_to_csv(df, filename, index=False):
    df.to_csv(filename, index=index)

# [RUN APP] 
def run_app():
    API_KEY = 'I3RTEm6vso7yOXBhGcYSidwUhRHaSgWy'
    # GET ALL ACTIVE STOCKS FOR US MARKET - XNAS (NASDAQ), XNYS (New York Stock Exchange)
    # SEE https://help.wealthsimple.com/hc/en-ca/articles/360056580834-Available-stocks-and-ETFs 
    
    ## GET ALL ACTIVE NASDAQ ETFs
    # nasdaq_etfs = get_all_active_stocks(API_KEY, ticker_type='ETF', market='stocks', exchange='XNAS')
    # save_to_csv(nasdaq_etfs, 'data\\NASDAQ_ETF_ACTIVE.csv')
    # nasdaq_etfs = load_csv('data\\NASDAQ_ETF_ACTIVE.csv')
    # print('[NASDAQ ETFs]')
    # display(nasdaq_etfs)

    ## GET ALL ACTIVE NYSE ETFs
    # nyse_etfs = get_all_active_stocks(API_KEY, ticker_type='ETF', market='stocks', exchange='XNYS')
    # save_to_csv(nyse_etfs, 'data\\NYSE_ETF_ACTIVE.csv')
    # nyse_etfs = load_csv('data\\NYSE_ETF_ACTIVE.csv')
    # print('[NYSE ETFs]')
    # display(nyse_etfs)

    ## GET ALL ACTIVE NASDAQ STOCKS
    # nasdaq_stocks = get_all_active_stocks(API_KEY, ticker_type='CS', market='stocks', exchange='XNAS')
    # save_to_csv(nasdaq_stocks, 'data\\NASDAQ_STOCKS_ACTIVE.csv')
    # nasdaq_stocks = load_csv('data\\NASDAQ_STOCKS_ACTIVE.csv')
    # display(nasdaq_stocks)

    ## GET ALL ACTIVE NYSE STOCKS
    # nyse_stocks = get_all_active_stocks(API_KEY, ticker_type='CS', market='stocks', exchange='XNYS')
    # save_to_csv(nyse_stocks, 'data\\NYSE_STOCKS_ACTIVE.csv')
    # nyse_stocks = load_csv('data\\NYSE_STOCKS_ACTIVE.csv')
    # display(nyse_stocks)

    # # GET ALL DIVIDEND STOCKS
    dividend_stocks = get_all_dividend_stocks(API_KEY)
    # save_to_csv(dividend_stocks, 'data\DIVIDEND_STOCKS.csv')
    dividend_stocks = load_csv('data\dividend_stocks.csv')
    display(dividend_stocks)
    # Grouping by 'frequency' and counting the number of symbols
    frequency_count = dividend_stocks.groupby('frequency')['ticker'].count()
    print('[FREQUENCY COUNT]', frequency_count,'', sep='\n')
    # # Filtering for monthly paying stocks
    # dividend_stocks = dividend_stocks.query('frequency == 12')
    # display(dividend_stocks)
    # # Grouping by 'frequency' and counting the number of symbols
    # frequency_count = dividend_stocks.groupby('frequency')['ticker'].count()
    # print('[FREQUENCY COUNT AFTER FILTERING FOR MONTHLY PAYING STOCKS]', frequency_count,'', sep='\n')
    symbols_list = get_symbols_list(dividend_stocks['ticker'])
    print('[SYMBOLS LIST LENGTH]', len(symbols_list),'', sep='\n')
    
    # nasdaq_ticker_list = get_symbols_list(nasdaq_stocks['ticker'])
    # nyse_ticker_list = get_symbols_list(nyse_stocks['ticker'])
    # print('[NASDAQ TICKER LIST LENGTH]', len(nasdaq_ticker_list),'', sep='\n')
    # print('[NYSE TICKER LIST LENGTH]', len(nyse_ticker_list),'', sep='\n')
    ## COMBINE NASDAQ AND NYSE TICKER LISTS
    # combined_ticker_list = nasdaq_ticker_list + nyse_ticker_list
    # print('[COMBINED TICKER LIST LENGTH]', len(combined_ticker_list),'', sep='\n')
    
    data = get_stock_data(symbols_list, start_year=2000)
    # data.info()
    # remove_columns_with_few_data_points(data, min_data_points=len(data))
    # save_to_csv(data, 'data\\YAHOO_FINANCE_DATA.csv', index=True)
    # print('[DATA AFTER REMOVING COLUMNS WITH DATA POINTS LESS THAN THE LENGTH OF THE DATA]', len(data),'', sep='\n')
    # data.info()
    data = load_csv('data\\YAHOO_FINANCE_DATA.csv', has_date_column=True)
    # data.info()
    # display(data)
    
    data = data[['CALM', 'DGX', 'OZK', 'SLP', 'WSO', 'WST']]

    # INCREMENTAL OPTIMIZATION SECTION
    # RISK MODELS WE CAN USE TO OPTIMIZE THE PORTFOLIO
    rm = {'SAMPLE_COV': 0, 'EXP_COV': 1, 'SEMICOVARIANCE': 2, 'LEDOIT_WOLF': 3, 'LEDOIT_WOLF_CONSTANT_VARIANCE': 4, 'LEDOIT_WOLF_SINGLE_FACTOR': 5, 'LEDOIT_WOLF_CONSTANT_CORRELATION': 6, 'ORACLE_APPROXIMATING': 7}
    key, value = find_key_by_value(rm, 2)
    incremental_optimize_portfolio(data, initial_size=6, filename=f'data\\TOP_DIVIDEND_STOCKS_{key}', risk_model=value)
    
    ## LOADING THE BEST PORTFOLIO, DISPLAYING DISCRETE ALLOCATION AND VISUALIZING THE DATA SECTION 
    # semicovariance_weights = load_csv('data\\BEST_PORTFOLIO_DIVIDEND_STOCKS_SEMICOVARIANCE_18.71%.csv')
    # display(semicovariance_weights)
    # ## RENAME THE UNNAMED COLUMN TO TICKER
    # semicovariance_weights = semicovariance_weights.rename(columns={'Unnamed: 0': 'Ticker'})
    # ## CREATE A LIST OF SYMBOLS
    # symbols_list = get_symbols_list(semicovariance_weights['Ticker'])
    # benchmarks = get_stock_data()
    # ## DROP NA VALUES
    # benchmarks = benchmarks.dropna() 
    # ## FILTER THE DATA BY THE LIST OF TICKERS
    # semicovariance_prices = data[symbols_list]
    # ## MERGE THE BENCHMARKS WITH THE DATA
    # semicovariance_merged_df = pd.merge(benchmarks, semicovariance_prices, left_index=True, right_index=True)
    # display(semicovariance_merged_df)

    # ## SHOWING PORTFOLIO DISCRETE ALLOCATION
    # weights_dict = semicovariance_weights.set_index('Ticker').to_dict()['Weight']
    # portfolio_investment = 600
    # latest_prices = get_latest_prices(semicovariance_prices)    
    # da = DiscreteAllocation(weights_dict, latest_prices, total_portfolio_value=portfolio_investment)
    # allocation, leftover = da.lp_portfolio()
    # print(f"Discrete allocation performed with ${portfolio_investment:.2f}")
    # print(f"Leftover: ${leftover:.2f}")
    # ## create a dataframe of the allocation
    # allocation_df = pd.DataFrame.from_dict(allocation, orient='index', columns=['Shares'])
    # display(allocation_df)
    
    # VISUALIZING THE DATA
    visualize_data(semicovariance_merged_df)
     
    
if __name__ == '__main__':
    run_app()